In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(torch.cuda.get_arch_list())

print(f"Is CUDA supported by this system?{torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
       
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from brevitas.nn import QuantConv2d, QuantReLU, QuantLinear
from brevitas.quant.scaled_int import Int8ActPerTensorFloat
from brevitas.quant.scaled_int import Int8WeightPerTensorFloat
from brevitas.quant.scaled_int import Int8BiasPerTensorFloatInternalScaling


class MobileNetV2(nn.Module):
    def __init__(self):
        super(MobileNetV2, self).__init__()
        myWeight_bit_width = 4
        act_bit_width = 4

        self.inverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]
        self.conv1 = QuantConv2d(3, 32, kernel_size=3, stride=2, padding=1,  bias=True, input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,
                        weight_quant=Int8WeightPerTensorFloat, bias_quant=Int8BiasPerTensorFloatInternalScaling, return_quant_tensor=True,weight_bit_width=myWeight_bit_width)
        self.relu = QuantReLU(bit_width=act_bit_width, max_val=6, act_quant=Int8ActPerTensorFloat, 
                      input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,return_quant_tensor=True)
        self.features = self._make_layers(in_channels=32)
        self.conv_last = QuantConv2d(320, 1280, kernel_size=1,  bias=True, input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,
                        weight_quant=Int8WeightPerTensorFloat, bias_quant=Int8BiasPerTensorFloatInternalScaling, return_quant_tensor=True,weight_bit_width=myWeight_bit_width)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = QuantLinear(1280, 2,  bias=True, input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,
                        weight_quant=Int8WeightPerTensorFloat, bias_quant=Int8BiasPerTensorFloatInternalScaling, return_quant_tensor=True,weight_bit_width=myWeight_bit_width)

    def _make_layers(self, in_channels):
        layers = []
        for t, c, n, s in self.inverted_residual_setting:
            for i in range(n):
                if i == 0:
                    layers.append(QuantBottleneck(in_channels, c, s, t))
                else:
                    layers.append(QuantBottleneck(in_channels, c, 1, t))
                in_channels = c
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.features(x)
        x = self.conv_last(x)
        x = self.relu(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


class QuantBottleneck(nn.Module):
    def __init__(self, in_planes, out_planes, stride, expansion):
        super(QuantBottleneck, self).__init__()
        
        myWeight_bit_width = 4
        act_bit_width = 4

        self.stride = stride
        planes = expansion * in_planes
        self.conv1 = QuantConv2d(in_planes, planes, kernel_size=1, bias=True, input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,
                        weight_quant=Int8WeightPerTensorFloat, bias_quant=Int8BiasPerTensorFloatInternalScaling, return_quant_tensor=True,weight_bit_width=myWeight_bit_width)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu1 = QuantReLU(bit_width=act_bit_width, max_val=6, act_quant=Int8ActPerTensorFloat, 
                      input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,return_quant_tensor=True)
        self.conv2 = QuantConv2d(planes, planes, kernel_size=3, stride=stride, padding=1, groups=planes, bias=True,
                                  weight_quant=Int8WeightPerTensorFloat, 
                                  act_quant=Int8ActPerTensorFloat)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu2 = QuantReLU(bit_width=act_bit_width, max_val=6, act_quant=Int8ActPerTensorFloat, 
                      input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,return_quant_tensor=True)
        self.conv3 = QuantConv2d(planes, out_planes, kernel_size=1, bias=True, input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,
                        weight_quant=Int8WeightPerTensorFloat, bias_quant=Int8BiasPerTensorFloatInternalScaling, return_quant_tensor=True,weight_bit_width=myWeight_bit_width)
        self.bn3 = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        if stride == 1 and in_planes != out_planes:
            self.shortcut = nn.Sequential(
                QuantConv2d(in_planes, out_planes, kernel_size=1, stride=stride,  bias=True, input_quant=Int8ActPerTensorFloat,output_quant=Int8ActPerTensorFloat,
                        weight_quant=Int8WeightPerTensorFloat, bias_quant=Int8BiasPerTensorFloatInternalScaling, return_quant_tensor=True,weight_bit_width=myWeight_bit_width),
                nn.BatchNorm2d(out_planes),
            )

    def forward(self, x):
        out = self.relu1(self.bn1(self.conv1(x)))
        out = self.relu2(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x) if self.stride == 1 else out
        return out

In [13]:
model_path = "trained_models/mobilenetv2_w4a4_model.pth"
model = MobileNetV2()
model.load_state_dict(torch.load(model_path))
model.to(device)

MobileNetV2(
  (conv1): QuantConv2d(
    3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (fused_activation_quant_proxy): FusedActivationQuantProxy(
        (activation_impl): Identity()
        (tensor_quant): RescalingIntQuant(
          (int_quant): IntQuant(
            (float_to_int_impl): RoundSte()
            (tensor_clamp_impl): TensorClamp()
            (delay_wrapper): DelayWrapper(
              (delay_impl): _NoDelay()
            )
          )
          (scaling_impl): ParameterFromRuntimeStatsScaling(
            (stats_input_view_shape_impl): OverTensorView()
            (stats): _Stats(
              (stats_impl): AbsPercentile()
            )
            (restrict_scaling): _RestrictValue(
              (restrict_value_impl): FloatRestrictValue()
            )
            (clamp_scaling): _ClampValue(
              (clamp_min_ste): ScalarClampMinSte()
            )

In [ ]:
for name, param in model.named_parameters():
     print(f"Layer: {name}, Type: {param.dtype}, Size: {param.nelement()*param.element_size()} bytes")

In [14]:
from finn.util.visualization import showSrc, showInNetron
from finn.util.basic import make_build_dir
import os
    
build_dir = "finn_build_dir"
onnx_dir = "onnx_models"

ModuleNotFoundError: No module named 'finn'

In [ ]:
import onnx
import torch
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

model_for_export = model.cpu()
ready_model_export_path = 'onnx_models/modbilenetv2-ready.onnx'
input_t = 
export_onnx(model_For_export,export_path = ready_model_export_path,input_t = input_t)

In [ ]:
showInNetron(onnx_dir+"/mobilenetv2-ready.onnx")